In [1]:
import numpy
import ipywidgets as widgets 
from ipywidgets import interact, interactive_output, interactive 
import yaml 

In [2]:
with open('dataviz_lab.yml') as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    lab_dict = yaml.load(file, Loader=yaml.FullLoader)

    print(lab_dict)


{'q1': {'question': 'What are the three components of a data story?', 'options': {'a': 'Questions, Answers, Visualizations', 'b': 'EDA, Modelling and Visualizations', 'c': 'Questions, Data, and Visualizations'}, 'question_type': '0', 'answer': 'Questions, Data, and Visualizations'}, 'q2': {'question': 'what pandas command would you use to view the first 10 entries in a dataframe?', 'options': {'a': 'my_dataframe.tail()', 'b': 'my_dataframe.head(10)', 'c': 'my_dataframe.head(20)'}, 'question_type': '0', 'answer': 'my_dataframe.head(10)'}, 'q3': {'question': 'How much did Europe contribute to the World GDP in the 2007?', 'options': {'a': '33.4%', 'b': '25.5%', 'c': '35.6%', 'd': '1.0%'}, 'question_type': '0', 'answer': '33.4%'}, 'q4': {'question': 'What was the life expectancy of Swiss person in the year 1962 according to the animated plot. Pausing the animated plot may help?', 'options': {'a': '71.32 years', 'b': '69.45 years', 'c': '44.50 years'}, 'question_type': '0', 'answer': '71.32

In [3]:

def qtype1(options, qnum): 
    check_dict = {}

    for x in options.keys():
            key_name = str(qnum)+"_"+str(x)
            check_dict[key_name] = widgets.Checkbox(value=False,
                                             description=options[x],
                                             disabled=False,
                                             indent=False)    
    return check_dict

def qtype2(single_options, qnum2):
    output_widget = {}
    key_name = str(qnum2)+"_0"
    output_widget[key_name] = widgets.RadioButtons(
                                    options=list(single_options.values()),
                                #    value='pineapple', # Defaults to 'pineapple'
                                   layout={'width': 'max-content'}, # If the items' names are long
                                    description= " ",
                                    disabled=False
                                )

    return output_widget

        
def options_widgets(options, qnum, qtype): 
    
    all_ouput_widgets = {}
    if qtype == 0: 
        output_widget = qtype2(options, qnum)
        all_ouput_widgets.update(output_widget)
    elif qtype ==1: 
        output_widgets = qtype1(options, qnum)
        all_ouput_widgets.update(output_widgets)
    return all_ouput_widgets


def question_widget(question, select_crit): 
    question_string = "{}  ({})".format(question, select_crit)
    question_put = f"<b><font size='2'>{question_string}</b>"
    return widgets.HTML(value= question_put)

qkeys = list(lab_dict.keys())



all_widgets = []
new_widgets = {}
all_answers = []
for x in range(0, len(lab_dict)): 
        
        single_question = lab_dict[qkeys[x]]
        
        question = single_question["question"]
        options = single_question["options"]
        qtype = int(single_question["question_type"])
        all_answers.append(single_question["answer"])
        opt_widgets = options_widgets(options, x, qtype)
        old_widgets = opt_widgets
        new_widgets.update(old_widgets)
        
        
        qwidget = question_widget(question, qtype)
        all_widgets.append(qwidget)
        
        list_widgets = list(old_widgets.values())
        for x in list_widgets: 
            all_widgets.append(x)
        

# adding submit button 
submit_button = widgets.Button(description="Submit")
all_widgets.append(submit_button)


In [4]:
all_answers

['Questions, Data, and Visualizations',
 'my_dataframe.head(10)',
 '33.4%',
 '71.32 years',
 ['Stories are extremely effective when told using appropriate data and intuitive visualizations.',
  'Choosing the right visualization and selecting and filtering the right data for the visualization, are both crucial.']]

In [5]:


ui = widgets.VBox(all_widgets)


def f(**kwargs):

    return None

output = widgets.interactive_output(f, new_widgets)
display(ui, output)

store_data = []

def on_button_clicked(b):
    
    with output:
        store_data.clear()
        for key in new_widgets: 
            
            key_split = key.split("_")
            if key_split[1] == "0": 
                answer_value = key_split[0] +"_"+str(new_widgets[key].index) 
                store_data.append(new_widgets[key].value)

            if key_split[1].isalpha():
                store_data.append(new_widgets[key].description)
        print(store_data)

submit_button.on_click(on_button_clicked)



Output()

In [18]:
# get output from the widgets 
indicies = []
store_data = []
for key in new_widgets:             
    key_split = key.split("_")
    if key_split[1] == "0": 
        answer_value = key_split[0] +"_"+str(new_widgets[key].index) 
        store_data.append(new_widgets[key].value)
    
    if new_widgets[key].value is True: 
        store_data.append(new_widgets[key].description)
        
        
answer_array = store_data 


In [19]:
answer_array

['Questions, Answers, Visualizations',
 'my_dataframe.tail()',
 '33.4%',
 '71.32 years',
 'Stories are extremely effective when told using appropriate data and intuitive visualizations.']

In [25]:
all_answers_flattened = []
for x in all_answers:
    if type(x) is list: 
        for y in x: 
            all_answers_flattened.append(y)
    else: 
        all_answers_flattened.append(x)
    

['Questions, Data, and Visualizations',
 'my_dataframe.head(10)',
 '33.4%',
 '71.32 years',
 'Stories are extremely effective when told using appropriate data and intuitive visualizations.',
 'Choosing the right visualization and selecting and filtering the right data for the visualization, are both crucial.']